In [0]:
from pyspark.sql import functions as fn
from pyspark.sql.functions import col, lit
from pyspark.sql.types import *
from pyspark.sql import Window

In [0]:
df = spark.read.csv("s3://geekbrains-spark/sf-fire-calls.csv", header=True)

In [0]:
display(df.limit(5))

Call_Number,Unit_ID,Incident_Number,CallType,Call_Date_str,Watch_Date,Call_Final_Disposition,Available_DtTm,Address,City,Zipcode_of_Incident,Battalion,Station_Area,Box,OrigPriority,Priority,Final_Priority,ALS_Unit,Call_Type_Group,NumAlarms,UnitType,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhood,Location,RowID,Delay,call_date,call_date_year,call_date_month,call_date_day
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,null,1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333333333333,2002-11-01,2002,11,1
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,null,1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333333333335,2002-11-01,2002,11,1
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05,2002-11-01,2002,11,1
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.316666666666667,2002-11-01,2002,11,1
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166666666666666,2002-11-01,2002,11,1


Какие типы вызовов случались в 2018? Перечислите все.

В каком месяце 2018 года было больше всего вызовов?

Из какого района (neighborhood) было больше всего вызовов в 2018?

Какие районы (neighborhoods) дольше всего ждали (delay) пожарных в 2018?

Какая неделя в 2018 году была самая активная по количеству вызовов?

Переименуем колонку с датой вызова и из нее сделаем колонки с дотой в формате datetime  и еще отдельно год, месяц, день

In [0]:
 df = df.withColumnRenamed("Call Date", "Call_Date_str")

In [0]:
df= df.withColumn("call_date", fn.to_date("Call_Date_str", "dd/MM/yyyy"))

In [0]:
df = df.withColumn("call_date_year", fn.year(df.call_date)) \
       .withColumn("call_date_month", fn.month(df.call_date)) \
       .withColumn("call_date_day", fn.dayofmonth(df.call_date))

In [0]:
display(df.limit(5))

Call_Number,Unit_ID,Incident_Number,CallType,Call_Date_str,Watch_Date,Call_Final_Disposition,Available_DtTm,Address,City,Zipcode_of_Incident,Battalion,Station_Area,Box,OrigPriority,Priority,Final_Priority,ALS_Unit,Call_Type_Group,NumAlarms,UnitType,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhood,Location,RowID,Delay,call_date,call_date_year,call_date_month,call_date_day
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,null,1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333333333333,2002-11-01,2002,11,1
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,null,1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333333333335,2002-11-01,2002,11,1
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05,2002-11-01,2002,11,1
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.316666666666667,2002-11-01,2002,11,1
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166666666666666,2002-11-01,2002,11,1


Убираем из имен колонок пробелы

In [0]:
df = df.select([fn.col(col_name).alias(col_name.replace(' ', '_')) for col_name in df.columns])

In [0]:
display(df.limit(5))

Call_Number,Unit_ID,Incident_Number,CallType,Call_Date_str,Watch_Date,Call_Final_Disposition,Available_DtTm,Address,City,Zipcode_of_Incident,Battalion,Station_Area,Box,OrigPriority,Priority,Final_Priority,ALS_Unit,Call_Type_Group,NumAlarms,UnitType,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhood,Location,RowID,Delay,call_date,call_date_year,call_date_month,call_date_day
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,null,1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333333333333,2002-11-01,2002,11,1
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,null,1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333333333335,2002-11-01,2002,11,1
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05,2002-11-01,2002,11,1
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.316666666666667,2002-11-01,2002,11,1
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166666666666666,2002-11-01,2002,11,1


сохраняем в паркет

In [0]:
df.write.parquet("./firecalls.parquet")

In [0]:
parq_df = spark.read.parquet("dbfs:/firecalls.parquet/")

In [0]:
display(parq_df.limit(5))

Call_Number,Unit_ID,Incident_Number,CallType,Call_Date_str,Watch_Date,Call_Final_Disposition,Available_DtTm,Address,City,Zipcode_of_Incident,Battalion,Station_Area,Box,OrigPriority,Priority,Final_Priority,ALS_Unit,Call_Type_Group,NumAlarms,UnitType,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhood,Location,RowID,Delay,call_date,call_date_year,call_date_month,call_date_day
162962502,E08,16117863,Medical Incident,10/22/2016,10/22/2016,Code 2 Transport,10/22/2016 04:20:56 PM,2500 Block of 3RD ST,San Francisco,94107,B10,25,2533,2,3,3,true,Potentially Life-Threatening,1,ENGINE,2,10,10,Potrero Hill,"(37.7566800066504, -122.387999428154)",162962502-E08,4.916666666666667,null,null,null,null
162962502,E25,16117863,Medical Incident,10/22/2016,10/22/2016,Code 2 Transport,10/22/2016 04:34:57 PM,2500 Block of 3RD ST,San Francisco,94107,B10,25,2533,2,3,3,false,Potentially Life-Threatening,1,ENGINE,3,10,10,Potrero Hill,"(37.7566800066504, -122.387999428154)",162962502-E25,5.566666666666666,null,null,null,null
162962560,55,16117864,Medical Incident,10/22/2016,10/22/2016,Code 2 Transport,10/22/2016 05:31:45 PM,700 Block of GEARY ST,San Francisco,94109,B04,03,1556,3,3,3,true,Potentially Life-Threatening,1,MEDIC,2,4,6,Tenderloin,"(37.786469251596, -122.415638875478)",162962560-55,0.45,null,null,null,null
162962560,E03,16117864,Medical Incident,10/22/2016,10/22/2016,Code 2 Transport,10/22/2016 04:39:50 PM,700 Block of GEARY ST,San Francisco,94109,B04,03,1556,3,3,3,true,Potentially Life-Threatening,1,ENGINE,1,4,6,Tenderloin,"(37.786469251596, -122.415638875478)",162962560-E03,1.2333333333333334,null,null,null,null
162962575,54,16117865,Medical Incident,10/22/2016,10/22/2016,Code 2 Transport,10/22/2016 06:41:01 PM,500 Block of 15TH AVE,San Francisco,94118,B07,31,7154,2,2,2,true,Non Life-threatening,1,MEDIC,3,7,1,Outer Richmond,"(37.7777197046394, -122.473834612431)",162962575-54,2.1333333333333333,null,null,null,null


In [0]:
%fs ls /

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/firecalls.parquet/,firecalls.parquet/,0
